In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
plt.rc('font', family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)
import matplotlib.font_manager as fm
font_path = 'C:\Windows\Fonts\HMFMPYUN.TTF'
font_prop = fm.FontProperties(fname=font_path, size=20)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras import models
import seaborn as sns
import tensorflow.keras.utils as utils
from tensorflow.keras.callbacks import Callback

In [ ]:
seoul_2016=pd.read_csv('data/2016년 서울시 교통사고.csv', encoding='cp949')
seoul_2017=pd.read_csv('data/2017년 서울시 교통사고.csv', encoding='cp949')
seoul_2018=pd.read_csv('data/2018년 서울시 교통사고.csv', encoding='cp949')
seoul_2019=pd.read_csv('data/2019년 서울시 교통사고.csv', encoding='cp949')
seoul_2020=pd.read_csv('data/2020년 서울시 교통사고.csv', encoding='cp949')

In [ ]:
seoul=pd.concat([seoul_2016,seoul_2017,seoul_2018,seoul_2019,seoul_2020])

In [ ]:
seoul.index=range(seoul.shape[0])

In [ ]:
seoul.info()

In [ ]:
seoul.columns

In [ ]:
seoul[seoul.columns[0]].value_counts()

In [ ]:
seoul[seoul.columns[1]].value_counts()

In [ ]:
seoul[seoul.columns[2]].value_counts()

In [ ]:
seoul[seoul.columns[3]].value_counts()

In [ ]:
seoul[seoul.columns[4]].value_counts()

In [ ]:
seoul[seoul.columns[5]].value_counts()

In [ ]:
seoul[seoul.columns[6]].value_counts()

In [ ]:
seoul[seoul.columns[7]].value_counts()

In [ ]:
seoul[seoul.columns[8]].value_counts()

In [ ]:
seoul[seoul.columns[9]].value_counts()

In [ ]:
seoul[seoul.columns[10]].value_counts()

In [ ]:
seoul[seoul.columns[11]].value_counts()

In [ ]:
seoul[seoul.columns[12]].value_counts()

In [ ]:
seoul[seoul.columns[13]].value_counts()

In [ ]:
seoul[seoul.columns[14]].value_counts()

In [ ]:
seoul[seoul.columns[15]].value_counts()

In [ ]:
seoul[seoul.columns[16]].value_counts()

In [ ]:
seoul[seoul.columns[17]].value_counts()

In [ ]:
seoul[seoul.columns[18]].value_counts()

In [ ]:
seoul[seoul.columns[19]].value_counts()

In [ ]:
seoul[seoul.columns[20]].value_counts()

In [ ]:
seoul[seoul.columns[21]].value_counts()

In [ ]:
seoul[seoul.columns[3]].value_counts().index[0]

In [ ]:
yeoksam=seoul[seoul[seoul.columns[3]]==seoul[seoul.columns[3]].value_counts().index[0]]['사고번호']

In [ ]:
yeoksam.index=range(yeoksam.shape[0])

In [ ]:
int(yeoksam[0]/100000000)

In [ ]:
yeoksam=yeoksam.map(lambda x : int(x/100000000))

In [ ]:
yeoksam_num=yeoksam.to_frame().groupby('사고번호')['사고번호'].count().to_frame()

In [ ]:
yeoksam_num=yeoksam_num['사고번호']

In [ ]:
max_yeoksam_num=max(yeoksam_num)
min_yeoksam_num=min(yeoksam_num)

In [ ]:
yeoksam_num

In [ ]:
def seq2dataset(num, window_size):
    dataset = []
    for i in range(len(num) - window_size):
        subset = num[i: (i + window_size + 1)]
        dataset.append(subset)
    return np.array(dataset)

In [ ]:
dataset=seq2dataset(yeoksam_num,4)

In [ ]:
x_train = dataset[:,:-1]
y_train = dataset[:,-1]
x_train.shape, y_train.shape

In [ ]:
dataset

In [ ]:
x_train = (x_train-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num)
x_train[:2]

In [ ]:
y_train=y_train-1
y_train = utils.to_categorical(y_train)
y_train.shape

In [ ]:
x_train=x_train.reshape(1562,4,1)

In [ ]:
one_hot_vec_size = y_train.shape[1]

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(LSTM(
    units=128,
    kernel_initializer='glorot_normal',
    bias_initializer='zero',
    batch_input_shape=(1, 4, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size, 
    kernel_initializer='glorot_normal',
    bias_initializer='zero',    
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(4000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(LSTM(
    units=128,
    batch_input_shape=(1, 4, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size,     
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(6000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(GRU(
    units=64,
    kernel_initializer='glorot_normal',
    bias_initializer='zero',
    batch_input_shape=(1, 4, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size, 
    kernel_initializer='glorot_normal',
    bias_initializer='zero',    
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(4000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

In [ ]:
seq_in = [0.25]
seq_out = seq_in
seq_in = (seq_in-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num)
for i in range(1562):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) # 샘플 수, 타입스텝 수, 속성 수
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)+1
    np.append(seq_out, idx)
    np.append(seq_in, (idx -min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num))
    seq_in.pop(0)

model.reset_states()

In [ ]:
x_train[0]

In [ ]:
seq_in = [3,2,1,5]
seq_out = seq_in
seq_in = [ (it-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num) for it in seq_in] 
for i in range(1562):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) 
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)+1
    seq_out.append(idx)
    seq_in.append((idx-min_yeoksam_num)/(max_yeoksam_num-min_yeoksam_num))
    seq_in.pop(0)

model.reset_states()

In [ ]:
plt.plot(np.array(seq_out[0:100]))

In [ ]:
plt.plot(np.array(yeoksam_num)[0:100])

In [ ]:
plt.plot(np.array(seq_out[0:50]))

In [ ]:
(y_train.argmax(axis=1)+1) # 1 1 4 3 3 1

In [ ]:
np.array(yeoksam_num)[0:30]

In [ ]:
np.array(seq_out[0:30])

In [ ]:
np.array(seq_out[0:50])

In [ ]:
#model.save('yeoksammodel.h5') #50개

In [ ]:
model=models.load_model('yeoksammodel.h5')